In [109]:
from bertviz import head_view
import torch
from medbert.features.mlm_plos_dataset import MLM_PLOS_Dataset
from medbert.features.embeddings import BertEmbeddings
from medbert.common import pytorch
import numpy as np

In [112]:
inputs = torch.load("..\\data\\tokenized\\example_data.pt")
vocab = torch.load("..\\data\\vocab\\example_data.pt")
inv_vocab = {v:k for k,v in vocab.items()}
model = torch.load("..\\models\\mlm_plos_pretrained\\test\\model.pt")
model.eval()
patient_num = 3
max_len = 50
dataset = MLM_PLOS_Dataset(inputs, vocab, max_len=max_len)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)
input = [d for i,d in enumerate(dataloader) if i<20][patient_num]
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
config = model.config
embeddings = BertEmbeddings(config=config)
# input = pytorch.batch_to_device(input, device)
embedding_output = embeddings(input['codes'], input['segments'])
# process
print(input['attention_mask'])
number_codes = input['attention_mask'].sum()
outputs = model(inputs_embeds=embedding_output, 
            attention_mask=input['attention_mask'], 
            labels=input['labels'],
            next_sentence_label=input['plos'],
            output_attentions=True) 
attention = outputs[-1] 
attention_new = []
for att in attention:
    att = att[:,:,:number_codes,:number_codes]
    attention_new.append(att)
attention_new = tuple(attention_new)
# attention = truncate_attention(attention, input['codes'])
input_tokens = [inv_vocab[i] for i in inputs['codes'][patient_num]]
head_view(attention_new, input_tokens)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])
tensor(20)


<IPython.core.display.Javascript object>